In [1]:
from dotenv import load_dotenv
load_dotenv('../secret.env')

True

## parse document

In [3]:
url = "https://careersatdoordash.com/blog/how-doordash-leverages-llms-for-better-search-retrieval/"

In [4]:
from langchain_community.document_loaders import AsyncHtmlLoader

loader = AsyncHtmlLoader(url)
docs = loader.load()

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  1.10it/s]


In [5]:
from langchain_community.document_transformers import MarkdownifyTransformer

md = MarkdownifyTransformer()
converted_docs = md.transform_documents(docs)

print(converted_docs[0].page_content)

How DoorDash leverages LLMs for better search retrieval - DoorDash

[Skip to content](#content)

[![DoorDash](https://careersatdoordash.com/wp-content/themes/external-job-board/theme/images/doordash.png)](https://careersatdoordash.com/)

* [Mission & Values](https://careersatdoordash.com/mission-and-values/)
* [Working at DoorDash](https://careersatdoordash.com/working-at-doordash/)
* [Belonging](https://careersatdoordash.com/belonging/)
* [Blogs](https://careersatdoordash.com/blog/)
  + [Career Blog](https://careersatdoordash.com/blog/)
  + [Engineering Blog](https://careersatdoordash.com/engineering-blog/)
* [Career Areas](https://careersatdoordash.com/career-areas/)
  + [Engineering](https://careersatdoordash.com/career-areas/engineering/)
  + [Design](https://careersatdoordash.com/career-areas/design/)
  + [DashMart](https://careersatdoordash.com/career-areas/dashmart/)
  + [Marketing](https://careersatdoordash.com/career-areas/marketing/)
  + [Sales](https://careersatdoordash.com/

In [6]:
document = docs[0].page_content

In [7]:
title = docs[0].metadata['title']
title

'How DoorDash leverages LLMs for better search retrieval - DoorDash'

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model='gemini-2.0-flash')

In [5]:
from langchain.prompts import HumanMessagePromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate

In [15]:
import re
from langchain_core.exceptions import OutputParserException
from langchain_core.output_parsers import BaseOutputParser

class MarkdownOutputParser(BaseOutputParser):
    def parse(self, output: str) -> str:
        cleaned_text = output.strip()
        if len(cleaned_text) > 0:
            pattern = r"^\s*```(?:\w+)?\n([\s\S]*?)\n```\s*$"
            match = re.match(pattern, cleaned_text)
            if match:
                return match.group(1)
        else:
            raise OutputParserException("Output is empty")
    
    @property
    def _type(self) -> str:
        return "markdown_output_parseer"

## Extract keywords

In [206]:
system_template = """You are helpful assistant."""

human_template = """
You should extract the keywords you find important from the document between the document xml tags. Output should be a list of keywords.
<document>{document}</document>
"""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
prompt

ChatPromptTemplate(input_variables=['document'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['document'], input_types={}, partial_variables={}, template='\nYou should extract the keywords you find important from the document between the document xml tags. Output should be a list of keywords.\n<document>{document}</document>\n'), additional_kwargs={})])

In [207]:
chain = prompt | llm

In [122]:
with open('output.md', 'r') as f:
    document = f.read()

In [208]:
invoke_args = {
    'document': document,
}

response = chain.invoke(invoke_args)

In [209]:
print(response)

- DNN models
- online performance
- offline performance
- DoorDash
- Ads ML team
- ranking models
- debugging framework
- scalable methodology
- feature serving consistency
- feature freshness
- real-time features
- model serving
- model architecture
- Multi-Task Multi-Label (MTML)
- Feature Generation Disparity
- Data Distribution Shift (Concept Drift)
- Model Serving Instability
- offline replay
- AUC benchmark
- Feature staleness
- Cached residuals
- Online Feature Store



In [210]:
with open("keywords.md", "w") as f:
    f.write(response)

## Extract Figures, Tables

In [20]:
system_template = """You are helpful assistant."""

human_template = """
You should extract the figures and tables from the document.
document:
{document}
Use xml syntax to format your output:
- Use <img src=url> to insert an image. Provide a caption for the image with <caption> tag for accessibility.
- Use <table> for alignment. Keep tables concise and structured.
Provide the figures with <figures> tag and provide the tables with <tables> tag. If there no any images or tables, include empty figures or tables tag.
Do not output any other content.
"""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
prompt

ChatPromptTemplate(input_variables=['document'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['document'], input_types={}, partial_variables={}, template='\nYou should extract the figures and tables from the document.\ndocument:\n{document}\nUse xml syntax to format your output:\n- Use <img src=url> to insert an image. Provide a caption for the image with <caption> tag for accessibility.\n- Use <table> for alignment. Keep tables concise and structured.\nProvide the figures with <figures> tag and provide the tables with <tables> tag. If there no any images or tables, include empty figures or tables tag.\nDo not output any other content.\n'), additional_kwargs={})])

In [21]:
chain = prompt | llm

In [22]:
invoke_args = {
    'document': document,
}

response = chain.invoke(invoke_args)

In [23]:
print(response)

```xml
<figures>
  <figure>
    <img src="https://careersatdoordash.com/wp-content/uploads/2024/11/kasia-derenda-1KDXAcyd7H0-unsplash-scaled-e1731978199415.jpg">
    <caption>Featured image for the blog post.</caption>
  </figure>
  <figure>
    <img src="https://careersatdoordash.com/wp-content/uploads/2024/11/image-1024x249.png">
    <caption>Figure 1: Diagram of the life of a document and the life of a query.</caption>
  </figure>
  <figure>
    <img src="https://careersatdoordash.com/wp-content/uploads/2024/11/Search-FKG-Entity-Linking-1024x1024.png">
    <caption>Figure 2: Using LLMs for query segmentation and entity linking</caption>
  </figure>
    <figure>
    <img src="https://lh7-rt.googleusercontent.com/docsz/AD_4nXdtCHiY5aoVhq85jnnBkzslLaWSibCVNNtXUT2dp74-yYh2-I0k2QQJquTg4YPKRJpGlX0EkYXkRcTacR5UHuVgs1BXhzSRXHasl4jIYlytDpi7ssfLRC5SZE580_gCt6eQk3ryOJz1WyhUD5L_-JYVQhRn?key=Emdn_dVkP7-sr0acaE6MGQ">
    <caption>Figure 3: Ranked list of food items in the “Popular Dishes” carouse

In [13]:
response = """```xml
<figures>
<figure class="wp-block-image aligncenter is-resized"><img src="https://lh7-rt.googleusercontent.com/docsz/AD_4nXdGouL9pDnn3LKih2h4mmEwcZGPIlQYOxX_M3itKMtBWq56qBttGbcEVh8MggWkFAZWSan6jJolWjFi9-nAENu0UFG0gIVKhz_5T6wWWokIpa0JrPpXj-K2xZN0Y1xYbMLL9VSV?key=2JKv1fZGhwooiun5eP7Ez3ot" alt="" style="width:960px;height:auto"/><figcaption class="wp-element-caption"><em>Figure 1: Restaurant Discovery Ads Ranking Deep Learning Milestones</em></figcaption></figure>
<figure class="wp-block-image aligncenter is-resized"><img src="https://lh7-rt.googleusercontent.com/docsz/AD_4nXeC8PfTbKQ_oN14mOK-aGt2JZczRZjb0ivbKiS_XkeAc-CTFuMLh99iQAVOlXNTTRtdp3rPB15k9jg43-_qJiJNrj2M8bNLBvBtIi256_LyR7_Zq6e5zddxZIo2tFBQqRBLqsf4?key=2JKv1fZGhwooiun5eP7Ez3ot" alt="" style="width:746px;height:auto"/><figcaption class="wp-element-caption"><em>Figure 2: Feature Distribution Online (red) vs Offline (blue)</em></figcaption></figure>
<figure class="wp-block-image aligncenter is-resized"><img src="https://lh7-rt.googleusercontent.com/docsz/AD_4nXfVu9h5JPpG3wnNU3x6zVw8rVqxankDAxsNyD2_QZlc-bn4gYjJtefjkV4pe0M58WsheHYsqgDNL_EEBHBdzzAFwH_M5CmumrYhRJJify_L0vzlXpm898hLl0EV4Yvf0MGfwSxarw?key=2JKv1fZGhwooiun5eP7Ez3ot" alt="" style="width:593px;height:auto"/></figure>
<figure class="wp-block-image aligncenter is-resized"><img src="https://lh7-rt.googleusercontent.com/docsz/AD_4nXcr0SIZWL1hl1VF1CBw29l9xYdHISxI6l3lwM0afEWGBLVcO4GCuVJBoZnwgko8VWzlTgFQUjxk6C6fhbPQtw4zj-NjCCHMZe6QbykCDs6UBWOFnBeTXhg0yp_yl3j2_igkny33ZQ?key=2JKv1fZGhwooiun5eP7Ez3ot" alt="" style="width:589px;height:auto"/><figcaption class="wp-element-caption"><em>Figure 3: Feature Staleness</em></figcaption></figure>
<figure class="wp-block-image aligncenter size-large is-resized"><img src="https://careersatdoordash.com/wp-content/uploads/2024/12/image-1024x565.png" alt="" class="wp-image-13500" style="width:626px;height:auto"/><figcaption class="wp-element-caption"><em>Figure 4: AUC Relative Changes on models trained by -1/2/3/4 day feature offsets</em></figcaption></figure>
</figures>

<tables>
<figure class="wp-block-table"><table class="has-fixed-layout"><tbody><tr><td><strong>Feature Category</strong></td><td><strong>Feature Data Type</strong></td><td><strong>Description</strong></td></tr><tr><td rowspan="2">Existing Features</td><td>Dense Features</td><td>Mostly consumer engagement features</td></tr><tr><td>Sequence Features</td><td>Consumer-engaged business/food/cuisine tag sequence &amp; contextual features</td></tr><tr><td>Newly Added Features</td><td>Dense Features</td><td>Consumer promotion-related featuresAdditional consumer engagement features</td></tr></tbody></table><figcaption class="wp-element-caption"><em>Table 1: Model for investigation Feature details</em></figcaption></figure>
<figure class="wp-block-table"><table class="has-fixed-layout"><thead><tr><th><strong>Eval Date Range</strong></th><th><strong>Model</strong></th><th><strong>Feature Generation</strong></th><th><strong>Online vs Offline</strong></th><th><strong>AUC</strong></th></tr></thead><tbody><tr><td rowspan="3">One Week’s data in September</td><td>Baseline (Prod Model)</td><td>Online Logging</td><td>Online</td><td>Baseline Value</td></tr><tr><td>New model</td><td>Online Logging</td><td>Online Shadow</td><td>-1.80%</td></tr><tr><td>New model</td><td>-1d offline join new added features</td><td>Offline Replay</td><td>+2.05%</td></tr><tr><td rowspan="2">One Week’s data in June</td><td>Baseline (Prod Model)</td><td>Online Logging</td><td>Offline</td><td>+0.77%</td></tr><tr><td>New model</td><td>-1d offline join new added features</td><td>Offline</td><td>+2.105%</td></tr></tbody></table><figcaption class="wp-element-caption"><em>Table 2: AUC Benchmarks for offline Reply</em></figcaption></figure>
<figure class="wp-block-table"><table class="has-fixed-layout"><tbody><tr><td><strong>Feature Name</strong></td><td><strong>Feature Aggregation Level</strong></td><td><strong>Feature Aggregation Time Window</strong></td><td><strong>Feature Staleness</strong></td><td><strong>Feature Missing Rate</strong></td><td><strong>% of cached residuals</strong></td></tr><tr><td>Feature 1</td><td>Consumer level</td><td>Past 1 year</td><td>-3d/-4d</td><td>2.77%</td><td>1.15%</td></tr><tr><td>Feature 2</td><td>&lt;Consumer, Store&gt; level</td><td>Past 3 month</td><td>-3d</td><td>76.20%</td><td>45.6%</td></tr><tr><td>Feature 3</td><td>&lt;Consumer, Store&gt; level</td><td>Past half-year</td><td>-3d</td><td>70.19%</td><td>23.6%</td></tr><tr><td>Feature 4</td><td>Consumer level</td><td>Past 1 year</td><td>-3d/-4d</td><td>2.77%</td><td>6.07%</td></tr><tr><td>Feature 5</td><td>Consumer level</td><td>Past 3 months</td><td>-3d</td><td>45.18%</td><td>4.56%</td></tr><tr><td>Feature 6</td><td>Consumer level</td><td>Past 3 months</td><td>-3d</td><td>76.19%</td><td>31.0%</td></tr><tr><td>Feature 7</td><td>Consumer level</td><td>Past 1 month</td><td>-2d</td><td>76.19%</td><td>31.0%</td></tr><tr><td>Feature 8</td><td>Store level</td><td>Past 3 months</td><td>-3d</td><td>0.92%</td><td>34.9%</td></tr><tr><td>Feature 9</td><td>Consumer level</td><td>Past 3 months</td><td>-2d</td><td>45.18%</td><td>4.55%</td></tr><tr><td>Feature 10</td><td>Store level</td><td>Past 1 day</td><td>-3d</td><td>23.50%</td><td>24.7%</td></tr></tbody></table><figcaption class="wp-element-caption">Table 3: Cache Residuals of the 10 most important added features</figcaption></figure>
<figure class="wp-block-table"><table class="has-fixed-layout"><tbody><tr><td><strong>Feature Name</strong></td><td><strong>Feature Aggregation Level</strong></td><td><strong>Feature Aggregation Time Window</strong></td><td><strong>% of entities change**</strong></td><td><strong>% of feature value mismatch***</strong></td></tr><tr><td>Feature 1</td><td>Consumer level</td><td>Past 1 year</td><td>0.0258%</td><td>9.69%</td></tr><tr><td>Feature 2</td><td>&lt;Consumer, Store&gt; level</td><td>Past 3 month</td><td>1.04%</td><td>5.2%</td></tr><tr><td>Feature 3</td><td>&lt;Consumer, Store&gt; level</td><td>Past half-year</td><td>3.36%</td><td>11.3%</td></tr><tr><td>Feature 4</td><td>Consumer level</td><td>Past 1 year</td><td>0.0258%</td><td>9.68%</td></tr><tr><td>Feature 5</td><td>Consumer level</td><td>Past 3 months</td><td>1.0%</td><td>8.37%</td></tr><tr><td>Feature 6</td><td>Consumer level</td><td>Past 3 months</td><td>1.04%</td><td>4.66%</td></tr><tr><td>Feature 7</td><td>Consumer level</td><td>Past 1 month</td><td>1.04%</td><td>4.66%</td></tr><tr><td>Feature 8</td><td>Store level</td><td>Past 3 months</td><td>0.09%</td><td>4.38%</td></tr><tr><td>Feature 9</td><td>Consumer level</td><td>Past 3 months</td><td>1.04%</td><td>4.78%</td></tr><tr><td>Feature 10</td><td>Store level</td><td>Past 1 day</td><td>0.436%</td><td>35.7%</td></tr></tbody></table><figcaption class="wp-element-caption"><em>Table 4: Cache Residuals of the 10 most important added features<br>**<strong> </strong>the percentage of entity_ids that did not show up in the previous data.<br>***the percentage of feature values that are different from the previous day.&nbsp;</em></figcaption></figure>
<figure class="wp-block-table"><table class="has-fixed-layout"><tbody><tr><td>Solution</td><td>Pros</td><td>Cons</td></tr><tr><td>Short-term</td><td>Reduces AUC discrepancy immediately</td><td>Does not address cached residuals</td></tr><tr><td>Long-term</td><td>Effectively resolves both cached residuals and feature staleness.Improves model generalization.</td><td>Has the trade-off between development speed and data accuracyRequires system stability improvements to support feature logging of larger traffic.</td></tr></tbody></table><figcaption class="wp-element-caption"><em>Table 5: Comparison between short-term and long-term solutions</em></figcaption></figure>
</tables>
```
"""

In [16]:
figures_and_tables = MarkdownOutputParser().parse(response)

In [17]:
figures = re.search(r'<figures>(.*?)</figures>', figures_and_tables,re.DOTALL).group(1).strip()

In [18]:
tables = re.search(r'<tables>(.*?)</tables>', figures_and_tables,re.DOTALL).group(1).strip()

## Keyword Summary

In [212]:
system_template = """You are helpful assistant."""

human_template = """
You should create a summary from the document between the document xml tags that corresponds to the keywords between the keywords xml tags, but be sure to include important information and place any relevant figures or tables alongside it.
Write in {language}.
<document>{document}</document>
<keywords>{keywords}</keywords>
<figures>{figures}</figures>
<tables>{tables}</tables>
"""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
prompt

ChatPromptTemplate(input_variables=['document', 'figures', 'keywords', 'language', 'tables'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['document', 'figures', 'keywords', 'language', 'tables'], input_types={}, partial_variables={}, template='\nYou should create a summary from the document between the document xml tags that corresponds to the keywords between the keywords xml tags, but be sure to include important information and place any relevant figures or tables alongside it.\nWrite in {language}.\n<document>{document}</document>\n<keywords>{keywords}</keywords>\n<figures>{figures}</figures>\n<tables>{tables}</tables>\n'), additional_kwargs={})])

In [213]:
chain = prompt | llm

In [211]:
with open('keywords.md', 'r') as f:
    keywords = f.read()

In [214]:
invoke_args = {
    'document': document,
    'keywords': keywords,
    'figures': figures,
    'tables': tables,
    'language': 'Korean',
}

response = chain.invoke(invoke_args)

In [215]:
with open("summary.md", "w") as f:
    f.write(response)

## Reflection

In [7]:
from typing import List
from pydantic import BaseModel, Field

class Reflection(BaseModel):
    missing: str = Field(description="Critique of what is missing")
    advisable: str = Field(description="Critique of what is helpful for better writing")
    superfluous: str = Field(description="Critique of what is superfluous")

class Research(BaseModel):
    """Provide reflection and then follow up with search queries to improve the writing."""

    reflection: Reflection = Field(description="Your reflection on the initial writing for summary.")
    search_queries: List[str] = Field(description="1-3 search queries for researching improvements to address the critique of your current writing.")

print('##### Reflect #####')
draft = """일반적인 목적의 검색 엔진을 사용하는 고객에게 데이터와 트래픽을 격리하는 것은 매우 중요한 요소이다. 이를 위해 DoorDash는 **검색 스택(Search Stack)**이라는 개념을 도입했다. 검색 스택은 특정 인덱스에 전념하는 검색 서비스의 모음으로, 하나의 검색 스택에서 발생한 문제가 다른 검색 스택에 영향을 미치지 않도록 설계되었다. 즉, 각 검색 스택은 독립적으로 운영되며, 특정 인덱스에 대한 빌드 및 쿼리 작업만 수행할 수 있다. 이러한 격리 구조는 한 검색 스택에서 예기치 않은 문제가 발생하더라도 다른 검색 스택의 서비스에는 영향을 주지 않도록 보장한다. 또한, 각 테넌트가 프로비저닝한 모든 리소스를 쉽게 파악하고 관리할 수 있도록 지원하여, 리소스 사용에 대한 투명성을 확보하고 책임 소재를 명확히 할 수 있다. 검색 스택은 테넌트의 인덱스 스키마와 서비스를 격리하는 데 매우 유용하며, 스키마 및 스택 구성 변경 시 이전 버전과의 호환성을 고려하지 않고도 손쉽게 변경할 수 있다는 장점을 제공한다. 이러한 유연성을 확보하기 위해 DoorDash는 **제어 평면(Control Plane)**이라는 오케스트레이션 서비스를 설계했다."""

reflection = []
search_queries = []
for _ in range(2):
    structured_llm = llm.with_structured_output(Research, include_raw=True)
    info = structured_llm.invoke(draft)
    if not info['parsed'] == None:
        parsed_info = info['parsed']
        print(parsed_info)
        break

##### Reflect #####
reflection=Reflection(missing="검색 스택의 격리 구조가 실제 문제 발생 시 어떤 이점을 제공하는지에 대한 구체적인 사례가 부족합니다. 예를 들어, 특정 검색 스택에서 장애가 발생했을 때 다른 스택은 어떻게 영향을 받지 않는지 설명하는 것이 필요합니다. 또한, '제어 평면'의 역할과 기능에 대한 추가 설명이 필요합니다.", advisable="문장이 전반적으로 훌륭하지만, '검색 스택'과 '제어 평면'이라는 핵심 용어에 대한 더 자세한 설명이나 예시를 추가하면 독자의 이해도를 높일 수 있습니다. 예를 들어, '검색 스택'이 실제로 어떻게 데이터와 트래픽을 격리하는지, 그리고 '제어 평면'이 구체적으로 어떤 기능을 수행하여 유연성을 확보하는지 설명하는 것이 좋습니다.", superfluous='문장이 전반적으로 간결하고 핵심 내용을 잘 전달하고 있으므로 불필요한 부분은 없습니다.') search_queries=['DoorDash 검색 스택 장애 사례', 'DoorDash 제어 평면 아키텍처', '검색 스택 격리 효과']


In [ ]:
print(parsed_info.reflection.missing)
print(parsed_info.reflection.advisable)
print(parsed_info.reflection.superfluous)

검색 스택의 격리 구조가 실제 문제 발생 시 어떤 이점을 제공하는지에 대한 구체적인 사례가 부족합니다. 예를 들어, 특정 검색 스택에서 장애가 발생했을 때 다른 스택은 어떻게 영향을 받지 않는지 설명하는 것이 필요합니다. 또한, '제어 평면'의 역할과 기능에 대한 추가 설명이 필요합니다.
문장이 전반적으로 훌륭하지만, '검색 스택'과 '제어 평면'이라는 핵심 용어에 대한 더 자세한 설명이나 예시를 추가하면 독자의 이해도를 높일 수 있습니다. 예를 들어, '검색 스택'이 실제로 어떻게 데이터와 트래픽을 격리하는지, 그리고 '제어 평면'이 구체적으로 어떤 기능을 수행하여 유연성을 확보하는지 설명하는 것이 좋습니다.
문장이 전반적으로 간결하고 핵심 내용을 잘 전달하고 있으므로 불필요한 부분은 없습니다.


## Review Draft

In [32]:
system_template = """You are a technical writer. You have been given a document to review. You need to make sure that the document is well-written and that the reader can understand it. If there is a problem, you should put the answer “Fail” in the answer xml tag and the problem in the error xml tag. If it passes, you should put the answer “Pass” in the answer xml tag and the problem in the error xml tag. Output should be respective xml tags."""

human_template = """
Here is the summary of the document:
<summary>{summary}</summary>

Here are the keywords extracted from the document:
<keywords>{keywords}</keywords>

<figures>{figures}</figures>

<tables>{tables}</tables>

Please review the document and provide feedback.
"""

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_prompt = HumanMessagePromptTemplate.from_template(human_template)

prompt = ChatPromptTemplate.from_messages([system_prompt, human_prompt])
prompt

ChatPromptTemplate(input_variables=['figures', 'keywords', 'summary', 'tables'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a technical writer. You have been given a document to review. You need to make sure that the document is well-written and that the reader can understand it. If there is a problem, you should put the answer “Fail” in the answer xml tag and the problem in the error xml tag. If it passes, you should put the answer “Pass” in the answer xml tag and the problem in the error xml tag. Output should be respective xml tags.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['figures', 'keywords', 'summary', 'tables'], input_types={}, partial_variables={}, template='\nHere is the summary of the document:\n<summary>{summary}</summary>\n\nHere are the keywords extracted from the document:\n<keywords>{keyword

In [34]:
chain = prompt | llm

In [30]:
with open("summary.md", "r") as f:
    summary = f.read()

with open("keywords.md", "r") as f:
    keywords = f.read()

In [35]:
invoke_args = {
    'summary': summary,
    'keywords': keywords,
    'figures': figures,
    'tables': tables,
}

response = chain.invoke(invoke_args)

In [36]:
print(response)

```xml
<answer>Pass</answer>
<error>No errors found. The document is well-written, clearly explains the problem, the methodology used to identify the root cause, and the proposed solutions. The use of figures and tables is effective in presenting data and results. The summary is also accurate and concise.</error>
```


In [ ]:
MarkdownOutputParser().parse(response)

'<answer>Pass</answer>\n<error>No errors found. The document is well-written, clearly explains the problem, the methodology used to identify the root cause, and the proposed solutions. The use of figures and tables is effective in presenting data and results. The summary is also accurate and concise.</error>'